In [1]:
import numpy as np
import pandas as pd
import json
import pickle
from keras.models import load_model
from keras_preprocessing import image

Using TensorFlow backend.


In [2]:
f = open('image_mappings.pkl', 'rb')
obs_to_img = pickle.load(f)

In [3]:
def predictions(model, img, mappings):
    p = model.predict(img)
    p = dict(zip(range(998), p[0]))
    
    pred_ = {mappings[key]: value for key, value in p.items()}
    pred = {key: value for key, value in sorted(pred_.items(), key=lambda item: item[1], reverse=True)}
    
    return pred

In [4]:
def make_pred(model, mappings):
    df_avg = pd.DataFrame()
    df_max = pd.DataFrame()
    __ = 0
    
    for obs_id, files in obs_to_img.items():
        total = len(files)
        __ += total
        pred_avg = None
        pred_max = None

        for file in files:
            img = image.load_img('../Dataset/test/images/' + file, target_size=(224, 224))
            img = image.img_to_array(img) / 255
            img = np.expand_dims(img, axis=0)

            pred = predictions(model, img, mappings)

            if pred_avg is None:
                pred_avg = pred
            else:
                pred_avg = {key: pred_avg.get(key,0) + pred.get(key,0) for key in pred_avg.keys()}

            if pred_max is None:
                pred_max = pred
            else:
                pred_max = {key: max(pred_max.get(key,0), pred.get(key,0)) for key in pred_max.keys()}

        pred_avg = {key: value / total for key, value in pred_avg.items()}
        pred_avg = {key: value for key, value in sorted(pred_avg.items(), key=lambda item: item[1], reverse=True)}
        pred_max = {key: value for key, value in sorted(pred_max.items(), key=lambda item: item[1], reverse=True)}

        _ = 1
        for key, value in pred_avg.items():
            df_avg = df_avg.append(pd.Series([obs_id, key, value, _]), ignore_index=True)
            _ += 1

        _ = 1
        for key, value in pred_max.items():
            df_max = df_max.append(pd.Series([obs_id, key, value, _]), ignore_index=True)
            _ += 1
            
        print('Obs: ' + obs_id + 'Files: '  + str(__))
            
    df_avg = df_avg.astype({3: int})
    df_max = df_max.astype({3: int})
            
    return df_avg, df_max

In [5]:
model = load_model('../Models/resnet50_retrain.h5')

f = open('../Mappings/class_mappings_resnet_50_retrain.json', 'r')
mappings = json.load(f)
mappings = {value: key for key, value in mappings.items()}

In [6]:
df_avg, df_max = make_pred(model, mappings)

Obs: 144Files: 2
Obs: 67371Files: 9
Obs: 208872Files: 22
Obs: 56Files: 30
Obs: 30674Files: 37
Obs: 223260Files: 41
Obs: 227829Files: 46
Obs: 245369Files: 49
Obs: 101Files: 63
Obs: 257770Files: 75
Obs: 429Files: 77
Obs: 592Files: 78
Obs: 112975Files: 87
Obs: 289Files: 90
Obs: 209391Files: 96
Obs: 229779Files: 100
Obs: 27906Files: 112
Obs: 248Files: 120
Obs: 689Files: 183
Obs: 157556Files: 187
Obs: 24Files: 205
Obs: 29889Files: 209
Obs: 102085Files: 222
Obs: 206925Files: 237
Obs: 239Files: 241
Obs: 112988Files: 247
Obs: 216Files: 254
Obs: 583Files: 258
Obs: 327Files: 269
Obs: 650Files: 278
Obs: 261Files: 279
Obs: 359Files: 289
Obs: 138172Files: 299
Obs: 227371Files: 303
Obs: 20Files: 311
Obs: 52Files: 321
Obs: 303Files: 327
Obs: 19Files: 340
Obs: 195449Files: 350
Obs: 355Files: 379
Obs: 683Files: 381
Obs: 84001Files: 385
Obs: 243Files: 403
Obs: 165Files: 407
Obs: 118624Files: 410
Obs: 92410Files: 422
Obs: 529Files: 437
Obs: 123751Files: 446
Obs: 88195Files: 466
Obs: 188879Files: 470
Obs:

Obs: 96Files: 2641
Obs: 15Files: 2643
Obs: 314Files: 2647
Obs: 182398Files: 2650
Obs: 172124Files: 2653
Obs: 606Files: 2656
Obs: 41Files: 2657
Obs: 178319Files: 2663
Obs: 146Files: 2666
Obs: 615Files: 2669
Obs: 91375Files: 2674
Obs: 690Files: 2676
Obs: 154Files: 2679
Obs: 37029Files: 2688
Obs: 533Files: 2689
Obs: 16Files: 2690
Obs: 394Files: 2697
Obs: 44777Files: 2704
Obs: 89Files: 2705
Obs: 188Files: 2710
Obs: 234018Files: 2716
Obs: 203Files: 2717
Obs: 8016Files: 2721
Obs: 468Files: 2723
Obs: 526Files: 2728
Obs: 732Files: 2730
Obs: 134038Files: 2736
Obs: 423Files: 2739
Obs: 344Files: 2740
Obs: 35Files: 2742
Obs: 95341Files: 2746
Obs: 132745Files: 2749
Obs: 129295Files: 2759
Obs: 144774Files: 2767
Obs: 600Files: 2768
Obs: 236350Files: 2770
Obs: 415Files: 2771
Obs: 598Files: 2773
Obs: 22Files: 2774
Obs: 566Files: 2785
Obs: 66654Files: 2789
Obs: 93203Files: 2793
Obs: 160Files: 2795
Obs: 115Files: 2796
Obs: 70Files: 2802
Obs: 157188Files: 2806
Obs: 21Files: 2807
Obs: 104Files: 2810
Obs: 5

In [7]:
df_avg.to_csv('../Submissions/ResNetAvg.csv', sep=';', index=False)
df_max.to_csv('../Submissions/ResNetMax.csv', sep=';', index=False)